In [ ]:
import os
import tweepy as tw
import pandas as pd
import pickle
from datetime import date 

In [ ]:
path1 = "rumor_detection_acl2017/twitter15/tree" 
path2 = "rumor_detection_acl2017/twitter16/tree"  

In [ ]:
punc = ["''", ",", "[]", "->", "-", ">", "[", "]", "'", "R", "O", "T"]

In [ ]:
dict_tweetns15 = {}
dict_tweetns16 = {}

In [ ]:
for file in os.listdir(path1):
    s = str(file)
    s = s.replace(".txt", "")
    if s not in dict_tweetns15.keys():
        dict_tweetns15[s] = []
    ff = path1 + "/" + str(file)
    t = []
    xt = set()
    with open(ff, 'r') as f1:
        for line in f1:
            line =  line.rstrip("\n")
            t.append(line)
    for e in t:
        for char in e:
            if char in punc:
                if char == ">":
                    e = e.replace(char, " ")
                else:
                    e = e.replace(char, "")
        e = e.strip()  
        l = e.split()
        if len(l) == 4:
            tup = (int(l[1]), float(l[3]))
            xt.add(tup)
        else:
            tup = (int(l[3]), float(l[5]))
            xt.add(tup)
    xtl = list(xt)
    xtl1 = sorted(xtl, key = lambda x: x[1])
    dict_tweetns15[s] = xtl1

In [ ]:
len(dict_tweetns15)

In [ ]:
for file in os.listdir(path2):
    s = str(file)
    s = s.replace(".txt", "")
    if s not in dict_tweetns16.keys():
        dict_tweetns16[s] = []
    ff = path2 + "/" + str(file)
    t = []
    xt = set()
    with open(ff, 'r') as f2:
        for line in f2:
            line =  line.rstrip("\n")
            t.append(line)
    for e in t:
        for char in e:
            if char in punc:
                if char == ">":
                    e = e.replace(char, " ")
                else:
                    e = e.replace(char, "")
        e = e.strip()  
        l = e.split()
        if len(l) == 4:
            tup = (int(l[1]), float(l[3]))
            xt.add(tup)
        else:
            tup = (int(l[3]), float(l[5]))
            xt.add(tup)
    xtl = list(xt)
    xtl1 = sorted(xtl, key = lambda x: x[1])
    dict_tweetns16[s] = xtl1

In [ ]:
len(dict_tweetns16)

In [ ]:
dict15_to_write = open("dict_tweetns15.pickle", "wb")
pickle.dump(dict_tweetns15, dict15_to_write)

In [ ]:
dict16_to_write = open("dict_tweetns16.pickle", "wb")
pickle.dump(dict_tweetns16, dict16_to_write)

In [ ]:
dict15_to_read = open("dict_tweetns15.pickle", "rb")
loaded_dict15 = pickle.load(dict15_to_read)
print(len(loaded_dict15))

In [ ]:
dict16_to_read = open("dict_tweetns16.pickle", "rb")
loaded_dict16 = pickle.load(dict16_to_read)
print(len(loaded_dict16))

In [ ]:
consumer_key = input()

In [ ]:
consumer_secret = input()

In [ ]:
access_token = input()

In [ ]:
access_token_secret = input()

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit = True)

In [ ]:
month_year = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11,
             'Dec': 12}

In [ ]:
dto = date(2021, 2, 12)

In [ ]:
dto

In [ ]:
path3 = "Twitter15_dict"

In [ ]:
for file in os.listdir(path3):
    s = str(file)
    s = s.replace(".txt", "")
    s = s.replace("dict", "")
    if s in loaded_dict15.keys():
        del loaded_dict15[s]

In [ ]:
len(loaded_dict15)

In [ ]:
count = 0
for tweet in loaded_dict15.keys():
    count = count + 1
    print(count)
    lid = loaded_dict15[tweet]
    listt = []
    for tups in lid:
        c = 0
        uid = tups[0]
        try:
            p = api.get_user(uid)
        except tw.TweepError as e:
            c = c + 1
            print("Error! User not found. " + str(tups[0]))
        if c == 0:
            time = tups[1]
            tup1 = ([], time)
            tup1[0].append(len(p._json['description']))
            tup1[0].append(len(p._json['screen_name']))
            tup1[0].append(p._json['followers_count'])
            tup1[0].append(p._json['friends_count'])
            tup1[0].append(p._json['statuses_count'])
            pdate = list(p._json['created_at'].split())
            d = pdate[2]
            if d[0] == '0':
                d = pdate[2].replace('0', "")
            d = int(d)
            m = month_year[pdate[1]]
            y = int(pdate[-1])
            age = int(((dto - date(y, m, d)).days)/365)
            tup1[0].append(age)
            x1 = p._json['verified']
            if x1 == 'True' or x1 == 'true':
                tup1[0].append(1)
            else:
                tup1[0].append(0)   
            x2 = p._json['geo_enabled']
            if x2 == 'True' or x2 == 'true':
                tup1[0].append(1)
            else:
                tup1[0].append(0)
            listt.append(tup1)
    filename = "dict" + tweet + ".txt"
    with open(os.path.join(path3, filename), 'w+') as fl:
        for items in listt:
            fl.write(str(items) + '\n') 
    fl.close()